<h1>Movie Recommendation

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
movies_df = pd.read_csv('movies.dat', sep='::', engine='python', header=None, names=['MovieID', 'Title', 'Genres'], encoding="ISO-8859-1")
movies_df.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
ratings_df = pd.read_csv('ratings.dat', sep='::', engine='python', header=None, names=["UserID", "MovieID", "Rating", "Timestamp"])
ratings_df.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [4]:
users_df = pd.read_csv('users.dat', sep='::', engine='python', header=None, names=["UserID", "Gender", "Age", "Occupation", "Zip-code"])
users_df.head()

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [5]:
movies_df.shape, ratings_df.shape, users_df.shape

((3883, 3), (1000209, 4), (6040, 5))

In [6]:
movies_df.isna().sum(), ratings_df.isna().sum(), users_df.isna().sum()

(MovieID    0
 Title      0
 Genres     0
 dtype: int64,
 UserID       0
 MovieID      0
 Rating       0
 Timestamp    0
 dtype: int64,
 UserID        0
 Gender        0
 Age           0
 Occupation    0
 Zip-code      0
 dtype: int64)

NO null values

In [7]:
merged_df = pd.merge(pd.merge(ratings_df, movies_df), users_df)

In [8]:
merged_df.head()

,UserID,MovieID,Rating,Timestamp,Title,Genres,Gender,Age,Occupation,Zip-code
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,F,1,10,48067
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical,F,1,10,48067
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance,F,1,10,48067
3,1,3408,4,978300275,Erin Brockovich (2000),Drama,F,1,10,48067
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy,F,1,10,48067


In [9]:
merged_df['Genres'] = merged_df['Genres'].str.replace("|", " ")
merged_df.head()

C:\Users\Dell\AppData\Local\Temp\ipykernel_6204\1065196892.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  merged_df['Genres'] = merged_df['Genres'].str.replace("|", " ")


,UserID,MovieID,Rating,Timestamp,Title,Genres,Gender,Age,Occupation,Zip-code
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,F,1,10,48067
1,1,661,3,978302109,James and the Giant Peach (1996),Animation Children's Musical,F,1,10,48067
2,1,914,3,978301968,My Fair Lady (1964),Musical Romance,F,1,10,48067
3,1,3408,4,978300275,Erin Brockovich (2000),Drama,F,1,10,48067
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation Children's Comedy,F,1,10,48067


In [10]:
merged_df["features"] = merged_df['Rating'].astype(str) + " " + merged_df['Genres'].astype(str) + " " + merged_df['Gender'].astype(str) + " " + merged_df['Age'].astype(str)

merged_df.head()

,UserID,MovieID,Rating,Timestamp,Title,Genres,Gender,Age,Occupation,Zip-code,features
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,F,1,10,48067,5 Drama F 1
1,1,661,3,978302109,James and the Giant Peach (1996),Animation Children's Musical,F,1,10,48067,3 Animation Children's Musical F 1
2,1,914,3,978301968,My Fair Lady (1964),Musical Romance,F,1,10,48067,3 Musical Romance F 1
3,1,3408,4,978300275,Erin Brockovich (2000),Drama,F,1,10,48067,4 Drama F 1
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation Children's Comedy,F,1,10,48067,5 Animation Children's Comedy F 1


Shuffeling the dataset and selecting first 20000 rows because the dataset is too big.

In [22]:
sample_df = merged_df.sample(n=20000, random_state=42)

# Reset the index 
sample_df = sample_df.reset_index(drop=True)

sample_df.head()


,UserID,MovieID,Rating,Timestamp,Title,Genres,Gender,Age,Occupation,Zip-code,features
0,3466,1968,5,967161282,"Breakfast Club, The (1985)",Comedy Drama,M,25,14,94306,5 Comedy Drama M 25
1,5437,1610,4,961366105,"Hunt for Red October, The (1990)",Action Thriller,M,25,11,55426,4 Action Thriller M 25
2,770,445,3,976182730,Fatal Instinct (1993),Comedy,M,18,4,98499,3 Comedy M 18
3,889,2696,2,975250320,"Dinner Game, The (Le Dîner de cons) (1998)",Comedy,M,45,20,10024,2 Comedy M 45
4,2203,2013,5,974604523,"Poseidon Adventure, The (1972)",Action Adventure,M,45,12,53718,5 Action Adventure M 45


Creating Vector

In [23]:
vectorizer = CountVectorizer()
movie_vectors = vectorizer.fit_transform(sample_df['features'])
movie_vectors.toarray()

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 1, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 1, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]], dtype=int64)

Preparing cosine similarity matrix

In [24]:
cos_sim = cosine_similarity(movie_vectors)

In [34]:
movie_name = "Toy Story (1995)"

movie_index = sample_df[sample_df.Title == movie_name].index[0]

movie_index

1128

In [36]:
similar_movies = list(enumerate(cos_sim[movie_index]))

similar_movies

[(0, 0.2886751345948129),
 (1, 0.0),
 (2, 0.7071067811865475),
 (3, 0.35355339059327373),
 (4, 0.0),
 (5, 0.0),
 (6, 0.5773502691896258),
 (7, 0.0),
 (8, 0.0),
 (9, 0.0),
 (10, 0.22360679774997896),
 (11, 0.2886751345948129),
 (12, 0.0),
 (13, 0.22360679774997896),
 (14, 0.35355339059327373),
 (15, 0.0),
 (16, 0.0),
 (17, 0.0),
 (18, 0.2886751345948129),
 (19, 0.5),
 (20, 0.0),
 (21, 0.0),
 (22, 0.2886751345948129),
 (23, 0.8164965809277261),
 (24, 0.35355339059327373),
 (25, 0.5773502691896258),
 (26, 0.0),
 (27, 0.2886751345948129),
 (28, 0.2886751345948129),
 (29, 0.2886751345948129),
 (30, 0.6123724356957946),
 (31, 0.0),
 (32, 0.2886751345948129),
 (33, 0.35355339059327373),
 (34, 0.0),
 (35, 0.0),
 (36, 0.0),
 (37, 0.0),
 (38, 0.2886751345948129),
 (39, 0.0),
 (40, 0.22360679774997896),
 (41, 0.0),
 (42, 0.22360679774997896),
 (43, 0.0),
 (44, 0.35355339059327373),
 (45, 0.2886751345948129),
 (46, 0.0),
 (47, 0.0),
 (48, 0.0),
 (49, 0.0),
 (50, 0.0),
 (51, 0.5773502691896258),
 (

Sorting similar movies in descending order

In [37]:
sorted_sim_mov = sorted(similar_movies, key = lambda x:x[1], reverse = True)

In [38]:
sorted_sim_mov

[(60, 1.0),
 (420, 1.0),
 (1128, 1.0),
 (1518, 1.0),
 (2864, 1.0),
 (3515, 1.0),
 (5011, 1.0),
 (5179, 1.0),
 (6251, 1.0),
 (6876, 1.0),
 (8695, 1.0),
 (8769, 1.0),
 (9634, 1.0),
 (9993, 1.0),
 (10183, 1.0),
 (10440, 1.0),
 (11131, 1.0),
 (11319, 1.0),
 (11677, 1.0),
 (13228, 1.0),
 (13311, 1.0),
 (13647, 1.0),
 (14084, 1.0),
 (14138, 1.0),
 (14259, 1.0),
 (16010, 1.0),
 (16935, 1.0),
 (17950, 1.0),
 (18169, 1.0),
 (18289, 1.0),
 (18332, 1.0),
 (18974, 1.0),
 (19098, 1.0),
 (19865, 1.0),
 (248, 0.8944271909999159),
 (350, 0.8944271909999159),
 (2099, 0.8944271909999159),
 (3582, 0.8944271909999159),
 (4815, 0.8944271909999159),
 (7057, 0.8944271909999159),
 (7422, 0.8944271909999159),
 (11359, 0.8944271909999159),
 (14972, 0.8944271909999159),
 (15452, 0.8944271909999159),
 (15770, 0.8944271909999159),
 (65, 0.8660254037844388),
 (301, 0.8660254037844388),
 (525, 0.8660254037844388),
 (1033, 0.8660254037844388),
 (1255, 0.8660254037844388),
 (1350, 0.8660254037844388),
 (1475, 0.866025

printing top 10 recommended/similar movies

In [45]:
movie_set = set()
for movie in sorted_sim_mov:
    movie_set.add(sample_df[sample_df.index == movie[0]]["Title"].values[0])
    
    if len(movie_set) >= 10:
        break

list(movie_set)

for movie in movie_set:
    print(movie)

American Tail, An (1986)
Toy Story 2 (1999)
Chicken Run (2000)
American Tail: Fievel Goes West, An (1991)
Aladdin (1992)
Aladdin and the King of Thieves (1996)
Rugrats Movie, The (1998)
Adventures of Rocky and Bullwinkle, The (2000)
Toy Story (1995)
Bug's Life, A (1998)
